In [2]:
!conda install -y git
!pip install -q git+https://github.com/huggingface/transformers.git


Channels:
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/teaching/anaconda3/envs/chatbot

  added / updated specs:
    - git


The following NEW packages will be INSTALLED:

  curl               pkgs/main/linux-64::curl-8.12.1-h251f7ec_0 
  gettext            pkgs/main/linux-64::gettext-0.21.0-hedfda30_2 
  git                pkgs/main/linux-64::git-2.45.2-pl5340h9abc3c3_1 
  perl               pkgs/main/linux-64::perl-5.38.2-0_h5eee18b_perl5 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
!pip install -q datasets

In [4]:
!pip install -q lightning wandb

In [5]:
from datasets import load_dataset

dataset = load_dataset("apoidea/pubtabnet-html")

/home/teaching/anaconda3/envs/chatbot/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/34 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'split', 'imgid', 'html', 'html_table'],
        num_rows: 500777
    })
    validation: Dataset({
        features: ['image', 'split', 'imgid', 'html', 'html_table'],
        num_rows: 9115
    })
})

In [7]:
example = dataset['train'][0]
image = example['image']
# let's make the image a bit smaller when visualizing
width, height = image.size
display(image.resize((int(width*0.3), int(height*0.3))))

In [8]:
# let's load the corresponding JSON dictionary (as string representation)
ground_truth = example['html']
print(ground_truth)

{'cells': [{'tokens': ['<b>', 'K', 'i', 'n', 'e', 't', 'i', 'c', ' ', 'p', 'a', 'r', 'a', 'm', 'e', 't', 'e', 'r', '</b>'], 'bbox': [5, 9, 66, 19]}, {'tokens': ['<b>', 'N', 'D', '</b>'], 'bbox': [104, 9, 118, 19]}, {'tokens': ['<b>', 'D', '</b>'], 'bbox': [204, 9, 212, 19]}, {'tokens': ['<b>', 'D', ' ', '+', ' ', 'd', 'n', '-', 'R', 'h', 'o', 'A', '</b>'], 'bbox': [304, 9, 351, 19]}, {'tokens': ['<b>', 'D', ' ', '+', ' ', 'd', 'n', '-', 'R', 'a', 'c', '1', '</b>'], 'bbox': [404, 9, 450, 19]}, {'tokens': ['V', 'm', 'a', 'x'], 'bbox': [5, 25, 25, 35]}, {'tokens': ['1', '9', '.', '6', ' ', '±', ' ', '0', '.', '7', '5'], 'bbox': [104, 25, 148, 35]}, {'tokens': ['2', '6', '.', '2', ' ', '±', ' ', '0', '.', '8', '6', '*'], 'bbox': [204, 25, 250, 35]}, {'tokens': ['3', '1', '.', '3', ' ', '±', ' ', '0', '.', '8', '8', '<sup>', '†', '</sup>'], 'bbox': [304, 25, 350, 35]}, {'tokens': ['2', '1', '.', '6', ' ', '±', ' ', '0', '.', '9'], 'bbox': [404, 25, 442, 35]}, {'tokens': ['K', "'", ' ', 'f',

In [9]:
from ast import literal_eval

literal_eval(ground_truth)['cells']

[{'tokens': ['<b>',
   'K',
   'i',
   'n',
   'e',
   't',
   'i',
   'c',
   ' ',
   'p',
   'a',
   'r',
   'a',
   'm',
   'e',
   't',
   'e',
   'r',
   '</b>'],
  'bbox': [5, 9, 66, 19]},
 {'tokens': ['<b>', 'N', 'D', '</b>'], 'bbox': [104, 9, 118, 19]},
 {'tokens': ['<b>', 'D', '</b>'], 'bbox': [204, 9, 212, 19]},
 {'tokens': ['<b>',
   'D',
   ' ',
   '+',
   ' ',
   'd',
   'n',
   '-',
   'R',
   'h',
   'o',
   'A',
   '</b>'],
  'bbox': [304, 9, 351, 19]},
 {'tokens': ['<b>',
   'D',
   ' ',
   '+',
   ' ',
   'd',
   'n',
   '-',
   'R',
   'a',
   'c',
   '1',
   '</b>'],
  'bbox': [404, 9, 450, 19]},
 {'tokens': ['V', 'm', 'a', 'x'], 'bbox': [5, 25, 25, 35]},
 {'tokens': ['1', '9', '.', '6', ' ', '±', ' ', '0', '.', '7', '5'],
  'bbox': [104, 25, 148, 35]},
 {'tokens': ['2', '6', '.', '2', ' ', '±', ' ', '0', '.', '8', '6', '*'],
  'bbox': [204, 25, 250, 35]},
 {'tokens': ['3',
   '1',
   '.',
   '3',
   ' ',
   '±',
   ' ',
   '0',
   '.',
   '8',
   '8',
   '<sup>',
 

In [10]:
from transformers import Pix2StructForConditionalGeneration, AutoProcessor

repo_id = "KennethTM/pix2struct-base-table2html"

processor = AutoProcessor.from_pretrained(repo_id)
model = Pix2StructForConditionalGeneration.from_pretrained(repo_id, is_encoder_decoder=True)

In [ ]:
for name, param in model.decoder.named_parameters():
    if "layer.0" in name or "layer.1" in name:
        param.requires_grad = False


model.shared.weight.requires_grad = False  # shared embedding for T5
model.encoder.embed_tokens.weight.requires_grad = False  # if exists
model.encoder.embed_positions.weight.requires_grad = False  # optional

In [34]:
import json
import random
import ast
from typing import Any, List
from torch.utils.data import Dataset
from tqdm import tqdm

added_tokens = []

class ImageCaptioningDataset(Dataset):
    def __init__(
        self,
        dataset_name_or_path: str,
        max_patches: int = 1024,
        max_length: int = 512,
        split: str = "train",
        ignore_id: int = -100,
        task_start_token: str = "",
        prompt_end_token: str = None,
        sort_json_key: bool = True,
    ):
        super().__init__()

        self.split = split
        print(f"Loading dataset from: {dataset_name_or_path}, split: {split}...")
        raw_dataset = load_dataset(dataset_name_or_path, split=self.split)
        self.dataset = raw_dataset.select(range(min(15000, len(raw_dataset))))
        print(f"Loaded {len(self.dataset)} samples (limited to 15000).")

        self.max_patches = max_patches
        self.max_length = max_length
        self.ignore_id = ignore_id
        self.task_start_token = task_start_token
        self.prompt_end_token = prompt_end_token if prompt_end_token else task_start_token
        self.sort_json_key = sort_json_key

        self.gt_token_sequences = []
        print("Processing HTML annotations and converting to token sequences...")
        for i, raw in enumerate(tqdm(self.dataset["html"])):
            try:
                gt = json.loads(raw)
            except json.JSONDecodeError:
                gt = ast.literal_eval(raw)

            if "gt_parses" in gt:
                gt_jsons = gt["gt_parses"]
            elif "html" in gt and isinstance(gt["html"], dict):
                gt_jsons = [gt["html"]]
            elif "cells" in gt and isinstance(gt["cells"], list):
                gt_jsons = [gt]
            else:
                raise ValueError(
                    f"Unrecognized ground_truth format, keys={list(gt.keys())}"
                )

            self.gt_token_sequences.append([
                self.json2token(
                    j,
                    update_special_tokens_for_json_key=(self.split == "train"),
                    sort_json_key=self.sort_json_key
                ) for j in gt_jsons
            ])

        self.add_tokens([self.task_start_token, self.prompt_end_token])
        self.prompt_end_token_id = processor.tokenizer.convert_tokens_to_ids(self.prompt_end_token)

    def json2token(self, obj: Any, update_special_tokens_for_json_key: bool = True, sort_json_key: bool = True):
        if type(obj) == dict:
            if len(obj) == 1 and "text_sequence" in obj:
                return obj["text_sequence"]
            else:
                output = ""
                if sort_json_key:
                    keys = sorted(obj.keys(), reverse=True)
                else:
                    keys = obj.keys()
                for k in keys:
                    if update_special_tokens_for_json_key:
                        self.add_tokens([fr"<s_{k}>", fr"</s_{k}>"])
                    output += (
                        fr"<s_{k}>"
                        + self.json2token(obj[k], update_special_tokens_for_json_key, sort_json_key)
                        + fr"</s_{k}>"
                    )
                return output
        elif type(obj) == list:
            return r"<sep/>".join(
                [self.json2token(item, update_special_tokens_for_json_key, sort_json_key) for item in obj]
            )
        else:
            obj = str(obj)
            if f"<{obj}/>" in added_tokens:
                obj = f"<{obj}/>"
            return obj

    def add_tokens(self, list_of_tokens: List[str]):
        new_tokens = [tok for tok in list_of_tokens if tok not in added_tokens]
        if not new_tokens:
            return
        newly_added_num = processor.tokenizer.add_tokens(new_tokens)
        if newly_added_num > 0:
            model.decoder.resize_token_embeddings(len(processor.tokenizer))
            added_tokens.extend(new_tokens)
            print(f"Added {newly_added_num} new tokens. Total added: {len(added_tokens)}")

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, idx):
        if idx % 1000 == 0:
            print(f"Processing item {idx}/{len(self.dataset)}")

        item = self.dataset[idx]

        # prepare inputs
        encoding = processor(images=item["image"], max_patches=self.max_patches, return_tensors="pt")
        encoding = {k:v.squeeze() for k,v in encoding.items()}

        # prepare targets
        target_sequence = random.choice(self.gt_token_sequences[idx])
        input_ids = processor.tokenizer(
            target_sequence,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        ).input_ids

        labels = input_ids.squeeze().clone()
        labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id
        encoding["labels"] = labels
        return encoding, target_sequence


In [35]:
train_dataset = ImageCaptioningDataset("apoidea/pubtabnet-html",
                                       split="train", sort_json_key=False) # cord dataset is preprocessed, so no need for this
val_dataset = ImageCaptioningDataset("apoidea/pubtabnet-html",
                                       split="validation", sort_json_key=False) # cord dataset is preprocessed, so no need for this

Loading dataset from: apoidea/pubtabnet-html, split: train...


Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/34 [00:00<?, ?it/s]

Loaded 15000 samples (limited to 15000).
Processing HTML annotations and converting to token sequences...


...

Loading dataset from: apoidea/pubtabnet-html, split: validation...


Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Loaded 9115 samples (limited to 15000).
Processing HTML annotations and converting to token sequences...


100%|██████████████████████████████████████████████████████████████████████████| 9115/9115 [00:28<00:00, 316.37it/s]


In [36]:
encoding, target_sequence = train_dataset[0]
print(encoding.keys())

Processing item 0/15000
dict_keys(['flattened_patches', 'attention_mask', 'labels'])


In [37]:
print(processor.decode([id.item() for id in encoding["labels"] if id != -100]))

<s_cells><s_tokens> <b><sep/>K<sep/>i<sep/>n<sep/>e<sep/>t<sep/>i<sep/>c<sep/> <sep/>p<sep/>a<sep/>r<sep/>a<sep/>m<sep/>e<sep/>t<sep/>e<sep/>r<sep/></b></s_tokens><s_bbox> 5<sep/>9<sep/>66<sep/>19</s_bbox> <sep/><s_tokens> <b><sep/>N<sep/>D<sep/></b></s_tokens><s_bbox> 104<sep/>9<sep/>118<sep/>19</s_bbox> <sep/><s_tokens> <b><sep/>D<sep/></b></s_tokens><s_bbox> 204<sep/>9<sep/>212<sep/>19</s_bbox> <sep/><s_tokens> <b><sep/>D<sep/> <sep/>+<sep/> <sep/>d<sep/>n<sep/>-<sep/>R<sep/>h<sep/>o<sep/>A<sep/></b></s_tokens><s_bbox> 304<sep/>9<sep/>351<sep/>19</s_bbox> <sep/><s_tokens> <b><sep/>D<sep/> <sep/>+<sep/> <sep/>d<sep/>n<sep/>-<sep/>R<sep/>a<sep/>c<sep/>1<sep/></b></s_tokens><s_bbox> 404<sep/>9<sep/>450<sep/>19</s_bbox> <sep/><s_tokens> V<sep/>m<sep/>a<sep/>x</s_tokens><s_bbox> 5<sep/>25<sep/>25<sep/>35</s_bbox> <sep/><s_tokens> 1<sep/>9<sep/>.<sep/>6<sep/> <sep/>±<sep/> <sep/>0<sep/>.<sep/>7<sep/>5</s_tokens><s_bbox> 104<sep/>25<sep/>148<sep/>35</s_bbox> <sep/><s_tokens> 2<sep/>6<sep/>

In [38]:
print(target_sequence)

<s_cells><s_tokens><b><sep/>K<sep/>i<sep/>n<sep/>e<sep/>t<sep/>i<sep/>c<sep/> <sep/>p<sep/>a<sep/>r<sep/>a<sep/>m<sep/>e<sep/>t<sep/>e<sep/>r<sep/></b></s_tokens><s_bbox>5<sep/>9<sep/>66<sep/>19</s_bbox><sep/><s_tokens><b><sep/>N<sep/>D<sep/></b></s_tokens><s_bbox>104<sep/>9<sep/>118<sep/>19</s_bbox><sep/><s_tokens><b><sep/>D<sep/></b></s_tokens><s_bbox>204<sep/>9<sep/>212<sep/>19</s_bbox><sep/><s_tokens><b><sep/>D<sep/> <sep/>+<sep/> <sep/>d<sep/>n<sep/>-<sep/>R<sep/>h<sep/>o<sep/>A<sep/></b></s_tokens><s_bbox>304<sep/>9<sep/>351<sep/>19</s_bbox><sep/><s_tokens><b><sep/>D<sep/> <sep/>+<sep/> <sep/>d<sep/>n<sep/>-<sep/>R<sep/>a<sep/>c<sep/>1<sep/></b></s_tokens><s_bbox>404<sep/>9<sep/>450<sep/>19</s_bbox><sep/><s_tokens>V<sep/>m<sep/>a<sep/>x</s_tokens><s_bbox>5<sep/>25<sep/>25<sep/>35</s_bbox><sep/><s_tokens>1<sep/>9<sep/>.<sep/>6<sep/> <sep/>±<sep/> <sep/>0<sep/>.<sep/>7<sep/>5</s_tokens><s_bbox>104<sep/>25<sep/>148<sep/>35</s_bbox><sep/><s_tokens>2<sep/>6<sep/>.<sep/>2<sep/> <sep/>±

In [39]:
print("Number of added tokens:", len(added_tokens))
print(added_tokens)

Number of added tokens: 0
[]


In [40]:
len(processor.tokenizer)

50352

In [41]:
from torch.utils.data import DataLoader
import multiprocessing

num_cores = multiprocessing.cpu_count()

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=num_cores)
val_dataloader = DataLoader(val_dataset, batch_size=4, num_workers=num_cores)

In [42]:
# get first batch
batch = next(iter(train_dataloader))
encoding, target_sequences = batch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [43]:
for k,v in encoding.items():
  print(k,v.shape)

flattened_patches torch.Size([4, 1024, 770])
attention_mask torch.Size([4, 1024])
labels torch.Size([4, 512])


In [23]:
!pip install nltk


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.4 MB/s eta 0:00:00


In [44]:
print(processor.batch_decode([id for id in encoding["labels"].squeeze().tolist() if id != -100]))

['<s_cells><s_tokens> <b><sep/>P<sep/>C<sep/>T<sep/></b></s_tokens><s_bbox> 41<sep/>5<sep/>61<sep/>14</s_bbox> <sep/><s_tokens> <b><sep/>R<sep/><sup><sep/>2<sep/></sup><sep/></b></s_tokens><s_bbox> 108<sep/>4<sep/>119<sep/>14</s_bbox> <sep/><s_tokens> <b><sep/>N<sep/>e<sep/>e<sep/>d<sep/>s<sep/> <sep/>i<sep/>n<sep/>d<sep/>i<sep/>c<sep/>a<sep/>t<sep/>o<sep/>r<sep/></b></s_tokens><s_bbox> 166<sep/>5<sep/>229<sep/>14</s_bbox> <sep/><s_tokens> <b><sep/>S<sep/>t<sep/>a<sep/>n<sep/>d<sep/>a<sep/>r<sep/>d<sep/>i<sep/>s<sep/>e<sep/>d<sep/> <sep/>b<sep/>e<sep/>t<sep/>a<sep/> <sep/>c<sep/>o<sep/>e<sep/>f<sep/>f<sep/>i<sep/>c<sep/>i<sep/>e<sep/>n<sep/>t<sep/></b></s_tokens><s_bbox> 279<sep/>5<sep/>393<sep/>14</s_bbox> <sep/><s_tokens> <b><sep/>%<sep/> <sep/>v<sep/>a<sep/>r<sep/>i<sep/>a<sep/>n<sep/>c<sep/>e<sep/> <sep/>e<sep/>x<sep/>p<sep/>l<sep/>a<sep/>i<sep/>n<sep/>e<sep/>d<sep/></b></s_tokens><s_bbox> 411<sep/>5<sep/>492<sep/>14</s_bbox> <sep/><s_tokens> <b><sep/>P<sep/>C<sep/>T<sep/>1<sep/> <

In [45]:
from pathlib import Path
import re
from nltk import edit_distance
import numpy as np
import wandb

import torch

from transformers.optimization import Adafactor, get_cosine_schedule_with_warmup

import pytorch_lightning as pl


class Pix2Struct(pl.LightningModule):
    def __init__(self, config, processor, model):
        super().__init__()
        self.config = config
        self.processor = processor
        self.model = model

    def training_step(self, batch, batch_idx):
        encoding, _ = batch

        outputs = self.model(**encoding)
        loss = outputs.loss
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = Adafactor(self.parameters(), scale_parameter=False, relative_step=False, lr=self.config.get("lr"), weight_decay=1e-05)
        scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.config.get("num_warmup_steps"),
                                                    num_training_steps=self.config.get("max_steps"))

        return [optimizer], [scheduler]

    def train_dataloader(self):
        return train_dataloader


In [46]:
config = {
          "num_warmup_steps": 1000,

          "lr": 0.01,
          "check_val_every_n_epoch": 5,
          "gradient_clip_val": 1.0,
          "warmup_steps": 300, # 800/8*30/10, 10%
          "accumulate_grad_batches": 8,
          "verbose": True,
          }

pl_module = Pix2Struct(config, processor, model)

In [ ]:
for i in range(10):
    import wandb
    from pytorch_lightning.loggers import WandbLogger
    from pytorch_lightning.callbacks import Callback, EarlyStopping

    wandb.finish()
    wandb_logger = WandbLogger(project=f"Pix2StructStage{i}", name="demo-run-pix2struct-adafactor")



    trainer = pl.Trainer(
        accelerator="gpu",
        devices=1,
        max_epochs=3,
        check_val_every_n_epoch=0,   # Disable validation
        gradient_clip_val=config.get("gradient_clip_val"),  # Use gradient clipping
        accumulate_grad_batches=config.get("accumulate_grad_batches"),  # Use gradient accumulation
        logger=wandb_logger,         # Use the Weights and Biases logger
        # Add the checkpoint callback
    )
    trainer.fit(pl_module)
    final_save_dir = f"/home/teaching/Desktop/Grp_22/DL-hackathon/FinalFineTuning/Fine tuned data{i}/"
    pl_module.model.save_pretrained(final_save_dir)
    processor.save_pretrained(final_save_dir)

    print(f" Final model and processor saved to ⁠ {final_save_dir} ⁠")
    processor = AutoProcessor.from_pretrained(final_save_dir)
    model    = Pix2StructForConditionalGeneration.from_pretrained(final_save_dir)
    pl_module = Pix2Struct(config, processor, model)
    # 2) Move to GPU if available
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #modelF.to(device)
    

/home/teaching/anaconda3/envs/chatbot/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/teaching/anaconda3/envs/chatbot/lib/python3.10 ...
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                               | Params | Mode
--------------------------------------------------------------------
0 | model | Pix2StructForConditionalGeneration | 282 M  | eval
--------------------------------------------------------------------
282 M     Trainable params
0         Non-trainable params
282 M     Total params
1,129.806 Total estimated model params size (MB)
0         Modules in train mode
497       Modules in eval mode
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you c

Training: |                                                                                   | 0/? [00:00<?, …

Processing item 0/15000
Processing item 13000/15000
Processing item 5000/15000
Processing item 10000/15000
Processing item 6000/15000
Processing item 9000/15000
Processing item 11000/15000
Processing item 12000/15000
Processing item 4000/15000
Processing item 7000/15000
Processing item 8000/15000


`Trainer.fit` stopped: `max_epochs=3` reached.


...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                               | Params | Mode
--------------------------------------------------------------------
0 | model | Pix2StructForConditionalGeneration | 282 M  | eval
--------------------------------------------------------------------
282 M     Trainable params
0         Non-trainable params
282 M     Total params
1,129.806 Total estimated model params size (MB)
0         Modules in train mode
497       Modules in eval mode
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you c

Training: |                                                                                   | 0/? [00:00<?, …

Processing item 1000/15000



Detected KeyboardInterrupt, attempting graceful shutdown ...
Process Process-1792:
Process Process-1772:
Process Process-1785:
Process Process-1773:
Process Process-1774:
Process Process-1779:
Process Process-1790:
Process Process-1781:
Process Process-1778:
Process Process-1786:
Process Process-1767:
Process Process-1771:
Process Process-1791:
Process Process-1777:
Process Process-1787:
Process Process-1783:
Process Process-1788:
Process Process-1784:
Process Process-1782:
Process Process-1775:
Process Process-1776:
Process Process-1789:
Process Process-1780:
Process Process-1768:
Process Process-1770:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/teaching/anaconda3/envs/chatbot/lib/python3.10/multiprocessing/process.py", line 317, in _boot

NameError: name 'exit' is not defined